In [13]:
from flair.models import RelationExtractor
#from flair.datasets import RE_ENGLISH_DRUGPROT
from flair.trainers import ModelTrainer
from flair.embeddings import WordEmbeddings
from torch.optim.adam import Adam
from flair.datasets.relation_extraction import CoNLLUCorpus
from flair.data import Corpus
from flair.embeddings import FlairEmbeddings, TransformerWordEmbeddings
import shutil


In [11]:
import flair, torch
flair.device = torch.device('cuda') 

In [46]:
flair.device

device(type='cuda')

In [82]:
DATA_PATH = r'C:\Users\akumar33\Documents\RE_ALL\RE_ALL'


In [48]:
flair.__version__
MAX_EPOCHS = 1
lr = 0.01
MODEL_PATH = r"C:\Users\akumar33\Documents\RE_ALL\RE_ALL\RE_ALL"



In [47]:
# init embedding
embedding_legalbert = TransformerWordEmbeddings( 'nlpaueb/legal-bert-base-uncased' , layers='-1', model_max_length=512, layer_mean=False)


In [95]:
import os
with open(os.path.join(DATA_PATH,TRAIN_FILE)) as file:
    txt = file.read()

In [97]:
import os
with open(os.path.join(DATA_PATH,TEST_FILE)) as file:
    txt = file.read()

In [99]:
import os
with open(os.path.join(DATA_PATH,DEV_FILE)) as file:
    txt = file.read()

In [52]:
a = txt.split('\n\n')

In [69]:
len(txt)

6958272

In [80]:
with open(r"C:\Users\akumar33\Documents\RE_ALL\RE_ALL\dev.txt", "a", encoding='utf8') as file:
    file.truncate(0)
    var2= txt.split('\n\n') 
    for i in var2:
        seg = (i.split('\n'))
        for j in seg:

            if 'relations' in j:
                if ';' in j:
                    file.write(i+'\n\n')

In [100]:
with open(r"C:\Users\akumar33\Documents\RE_ALL\RE_ALL\dev_ner.txt", "a", encoding='utf8') as file:
    file.truncate(0)
    var2= txt.split('\n\n') 
    for i in var2:
        seg = (i.split('\n'))
        for j in seg:

            if '#' not in j:
                file.write(j+'\n')
        file.write('\n')

In [80]:
with open(r"C:\Users\akumar33\Documents\RE_ALL\RE_ALL\train_ner.txt", "a", encoding='utf8') as file:
    file.truncate(0)
    var2= txt.split('\n\n') 
    for i in var2:
        seg = (i.split('\n'))
        for j in seg:

            if '#' not in j:
                file.write(j+'\n')
        file.write('\n')

In [ ]:
with open(r"C:\Users\akumar33\Documents\RE_ALL\RE_ALL\train.txt", "a", encoding='utf8') as file:
var= txt.split('\n')    
for i in range(len(var)):
    if 'relations' in var[i]:
        
        if ';' not in var[i]:

            print(var[i], var[i-1], var[i+1])


In [84]:
classifier

RelationExtractor(
  (loss_function): CrossEntropyLoss()
  (token_embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): 

In [85]:
classifier._get_state_dict().keys()

dict_keys(['state_dict', 'token_embeddings', 'label_dictionary', 'label_type', 'span_label_type', 'loss_weights', 'pooling_operation', 'dropout_value', 'use_entity_pairs'])

In [90]:
from flair.models import TextClassifier
from flair.data import Sentence
# from flair.data import Sentence
from flair.models import SequenceTagger
from flair.datasets import ColumnCorpus
from flair.data import Corpus
from flair.trainers import ModelTrainer
from flair.data import Corpus 
from flair.datasets import ColumnCorpus

from flair.models import SequenceTagger

from flair.trainers import ModelTrainer

from flair.embeddings import TokenEmbeddings

from flair.embeddings import WordEmbeddings

from flair.embeddings import StackedEmbeddings

from flair.embeddings import FlairEmbeddings

In [86]:
DATA_PATH = r'C:\Users\akumar33\Documents\RE_ALL\RE_ALL'


In [88]:
MODEL_PATH = r"C:\Users\akumar33\Documents\RE_ALL\RE_ALL\ner_model"

In [ ]:
embedding_legalbert = TransformerWordEmbeddings( 'nlpaueb/legal-bert-base-uncased' , layers='-1', model_max_length=512, layer_mean=False)

In [107]:
##THIS CELL IS USED FOR NER TRAIN WITH FILE TRAIN_NER, TEST_NER

# tag_dictionary.add_unk = True # To handle unknown tags, such as 5-attor
TRAIN_FILE = f'train_ner.txt'
TEST_FILE = f'test_ner.txt'
DEV_FILE = f'dev_ner.txt'    


# Define corpus for each group
COLUMNS = {0:'id', 1: 'text', 2:'ner'}

#Load the corpus for each group 
corpus: Corpus =ColumnCorpus(DATA_PATH, COLUMNS,
                            train_file=TRAIN_FILE,
                            test_file=TEST_FILE,
                            dev_file=DEV_FILE,
                            in_memory= False
                            )
#Create pletionary
# new tags of new corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')

tag_dictionary.add_item('O')
print(tag_dictionary)
# initialize new tagger with extended dictionary (re-use same embeddings as before)
tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,
        embeddings=embedding_legalbert,
        tag_dictionary=tag_dictionary,
        tag_type='ner',
    )

#Define the model
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train(MODEL_PATH, learning_rate=0.1,
                mini_batch_size=64,
                max_epochs=5, 
                monitor_train=True,
                monitor_test=True,
                checkpoint=True, 
                train_with_dev=True,
                embeddings_storage_mode= "gpu")



2021-11-30 04:59:19,137 Reading data from C:\Users\akumar33\Documents\RE_ALL\RE_ALL
2021-11-30 04:59:19,138 Train: C:\Users\akumar33\Documents\RE_ALL\RE_ALL\train_ner.txt
2021-11-30 04:59:19,139 Dev: C:\Users\akumar33\Documents\RE_ALL\RE_ALL\dev_ner.txt
2021-11-30 04:59:19,140 Test: C:\Users\akumar33\Documents\RE_ALL\RE_ALL\test_ner.txt
Dictionary with 17 tags: O, B-employmentTitle, I-employmentTitle, B-attorney, I-attorney, B-litigantRole, I-litigantRole, B-lawFirm, I-lawFirm, B-city, B-state, I-state, I-city, B-litigant, I-litigant, <START>, <STOP>
2021-11-30 04:59:25,969 ----------------------------------------------------------------------------------------------------
2021-11-30 04:59:25,972 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
 

In [109]:
tag_dictionary.get_items()

['O',
 'B-employmentTitle',
 'I-employmentTitle',
 'B-attorney',
 'I-attorney',
 'B-litigantRole',
 'I-litigantRole',
 'B-lawFirm',
 'I-lawFirm',
 'B-city',
 'B-state',
 'I-state',
 'I-city',
 'B-litigant',
 'I-litigant',
 '<START>',
 '<STOP>']